In [1]:
import os
import time
import shutil
import pathlib
import itertools
from PIL import Image

import cv2
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers

import warnings
warnings.filterwarnings("ignore")

print ('modules loaded')

modules loaded


In [3]:
data_path = "/kaggle/input/pv-potato/PV_Potato" 

images = []
labels = []

for subfolder in os.listdir(data_path):
    
    subfolder_path = os.path.join(data_path, subfolder)
    if not os.path.isdir(subfolder_path):
        continue
  
    for image_filename in os.listdir(subfolder_path):
        image_path = os.path.join(subfolder_path, image_filename)
        images.append(image_path)
    
        labels.append(subfolder)
 
data = pd.DataFrame({'image': images, 'label': labels})

In [4]:
strat = data['label']
train_df, dummy_df = train_test_split(data,  train_size= 0.80, shuffle= True, random_state= 123, stratify= strat)

strat = dummy_df['label']
valid_df, test_df = train_test_split(dummy_df,  train_size= 0.5, shuffle= True, random_state= 123, stratify= strat)

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 32
img_size = (150, 150)
channels = 3
img_shape = (img_size[0], img_size[1], channels)

tr_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

ts_gen = ImageDataGenerator()  


train_gen = tr_gen.flow_from_dataframe(
    train_df,
    x_col='image',
    y_col='label',
    target_size=img_size,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True,
    batch_size=batch_size
)

valid_gen = ts_gen.flow_from_dataframe(
    valid_df,
    x_col='image',
    y_col='label',
    target_size=img_size,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=False,
    batch_size=batch_size
)

test_gen = ts_gen.flow_from_dataframe(
    test_df,
    x_col='image',
    y_col='label',
    target_size=img_size,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=False,
    batch_size=batch_size
)

Found 7608 validated image filenames belonging to 3 classes.
Found 951 validated image filenames belonging to 3 classes.
Found 951 validated image filenames belonging to 3 classes.


In [7]:
from keras.applications import VGG16
from keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
import numpy as np

vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
vgg16_base.trainable = False

inputs = Input(shape=(150, 150, 3))
x = vgg16_base(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(3, activation='softmax')(x)
vgg16_model = Model(inputs, outputs)

vgg16_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy', 'AUC']  
)

vgg16_model.summary()

history = vgg16_model.fit(train_gen, epochs=10, validation_data=valid_gen)

test_loss, test_acc, test_auc = vgg16_model.evaluate(test_gen)

print(f"Test Accuracy: {test_acc}")
print(f"Test AUC: {test_auc}")

y_pred = vgg16_model.predict(test_gen)
y_pred_classes = np.argmax(y_pred, axis=1)

y_true = test_gen.classes  

print(classification_report(y_true, y_pred_classes))

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 150, 150, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ vgg16 (Functional)                   │ (None, 4, 4, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,780,739 (56.38 MB)

 Trainable params: 66,051 (258.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

Epoch 1/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 96s 363ms/step - AUC: 0.8060 - accuracy: 0.6529 - loss: 2.2656 - val_AUC: 0.9710 - val_accuracy: 0.8759 - val_loss: 0.3291
Epoch 2/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 49s 199ms/step - AUC: 0.9523 - accuracy: 0.8269 - loss: 0.4281 - val_AUC: 0.9844 - val_accuracy: 0.9180 - val_loss: 0.2319
Epoch 3/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 47s 192ms/step - AUC: 0.9680 - accuracy: 0.8673 - loss: 0.3425 - val_AUC: 0.9853 - val_accuracy: 0.9096 - val_loss: 0.2228
Epoch 4/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 47s 193ms/step - AUC: 0.9748 - accuracy: 0.8821 - loss: 0.3031 - val_AUC: 0.9876 - val_accuracy: 0.9253 - val_loss: 0.2075
Epoch 5/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 48s 195ms/step - AUC: 0.9794 - accuracy: 0.8966 - loss: 0.2702 - val_AUC: 0.9897 - val_accuracy: 0.9317 - val_loss: 0.1823
Epoch 6/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 62s 254ms/step - AUC: 0.9825 - accuracy: 0.8974 - loss: 0.2484 - val_AUC: 0.9901 - val_accuracy: 0.9369 - val_loss: 0.1817
Epoch 7/10
238/238 ━━━

In [9]:
from keras.applications import VGG19
from keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
import numpy as np

vgg19_base = VGG19(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
vgg19_base.trainable = False

inputs = Input(shape=(150, 150, 3))
x = vgg19_base(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(3, activation='softmax')(x)
vgg19_model = Model(inputs, outputs)

vgg19_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy', 'AUC']  
)

vgg19_model.summary()

history = vgg19_model.fit(train_gen, epochs=10, validation_data=valid_gen)

test_loss, test_acc, test_auc = vgg19_model.evaluate(test_gen)

print(f"Test Accuracy: {test_acc}")
print(f"Test AUC: {test_auc}")

y_pred = vgg19_model.predict(test_gen)
y_pred_classes = np.argmax(y_pred, axis=1)

y_true = test_gen.classes  

print(classification_report(y_true, y_pred_classes))


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)           │ (None, 150, 150, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ vgg19 (Functional)                   │ (None, 4, 4, 512)           │      20,024,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_3           │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,090,435 (76.64 MB)

 Trainable params: 66,051 (258.01 KB)

 Non-trainable params: 20,024,384 (76.39 MB)

Epoch 1/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 72s 282ms/step - AUC: 0.8190 - accuracy: 0.6724 - loss: 1.8669 - val_AUC: 0.9803 - val_accuracy: 0.9117 - val_loss: 0.2699
Epoch 2/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 77s 315ms/step - AUC: 0.9555 - accuracy: 0.8411 - loss: 0.4155 - val_AUC: 0.9883 - val_accuracy: 0.9390 - val_loss: 0.1922
Epoch 3/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 80s 324ms/step - AUC: 0.9720 - accuracy: 0.8720 - loss: 0.3209 - val_AUC: 0.9912 - val_accuracy: 0.9443 - val_loss: 0.1715
Epoch 4/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 72s 290ms/step - AUC: 0.9752 - accuracy: 0.8870 - loss: 0.2981 - val_AUC: 0.9930 - val_accuracy: 0.9506 - val_loss: 0.1531
Epoch 5/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 70s 289ms/step - AUC: 0.9780 - accuracy: 0.8966 - loss: 0.2795 - val_AUC: 0.9935 - val_accuracy: 0.9506 - val_loss: 0.1464
Epoch 6/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 66s 269ms/step - AUC: 0.9821 - accuracy: 0.9014 - loss: 0.2537 - val_AUC: 0.9938 - val_accuracy: 0.9590 - val_loss: 0.1336
Epoch 7/10
238/238 ━━━